In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import csv
import numpy as np

In [6]:
training_df = pd.read_csv('training.csv')
test_df = pd.read_csv('test.csv')
y_train = training_df['topic']
y_test = test_df['topic']

In [7]:
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.68


In [8]:
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = TfidfVectorizer(ngram_range=(1,2))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.642


In [9]:
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = TfidfVectorizer(ngram_range=(2,2))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.646


In [10]:
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = TfidfVectorizer(ngram_range=(1,5))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.636


In [11]:
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = TfidfVectorizer(ngram_range=(10,10))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.554


In [13]:
#might be good to get rid of irrelevant class...bias towards it especially since large class
#term or document frequency?
#a bit confused how test data is transformed...is term frequency within a doc calculated for test data
vectorizer = CountVectorizer()
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.728


In [14]:
vectorizer = CountVectorizer(ngram_range=(1,2))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.734


In [16]:
vectorizer = CountVectorizer(ngram_range=(1,3))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.732


In [15]:
vectorizer = CountVectorizer(ngram_range=(1,5))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.73


In [17]:
vectorizer = CountVectorizer(ngram_range=(1,1))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB(fit_prior=False)
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.73


In [18]:
vectorizer = CountVectorizer(ngram_range=(1,2))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB(fit_prior=False)
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.736


In [19]:
vectorizer = CountVectorizer(ngram_range=(1,3))
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB(fit_prior=False)
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.734


In [22]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), use_idf=False, sublinear_tf=True)
train_features = vectorizer.fit_transform(training_df['article_words'])
test_features = vectorizer.transform(test_df['article_words'])
classifier = MultinomialNB()
classifier.fit(train_features, y_train)
predictions = classifier.predict(test_features)
acc = np.mean(predictions == y_test)
print(acc)

0.6


In [ ]:
#work still needs to be done with the actual feature metric being used some combination of its frequency within a document and the differentiability of that word in a class compared to others...that might be what feature selection does

In [ ]:
#using feature selection